In [1]:
import os
import sqlite3
import pandas as pd
from skimage import measure, io
from skimage.morphology import label
import numpy as np

from explorer import get_crops

path = '/mnt/sextonlabdata/Active_Users_Data/Sophia/Colonoids/Caco2/ZO1Screen/SRM_BIS_009/'

con = sqlite3.connect(os.path.join(path, 'BIS_009.db'))

query_objects = """
SELECT *
FROM MyExpt_Per_Object_UMAP a
JOIN MyExpt_Per_Image b ON a.ImageNumber = b.ImageNumber
""" #change UMAP table name and column names accodingly
df = pd.read_sql_query(query_objects, con)

# Close the connection
con.close()

2025-06-23 15:51:07.536449: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-23 15:51:07.546989: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-23 15:51:07.550175: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-23 15:51:07.558412: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-23 15:51:08.275663: W tensorflow/compiler/tf2

In [2]:
bbox_col = 'Cell_AreaShape_BoundingBoxArea'
bbox_area = 46874

chan1_col = 'Image_FileName_DNA'
chan2_col = 'Image_FileName_CMO'
chan3_col = 'Image_FileName_Claudin2'

cluster_col = 'leiden_cluster'

base_path = "/mnt/instrumentdata/CV8000/Sophia/BIS009_20240701_102850/PECCU/"
output_path = "/hdd/tools/UMAP-Explorer/notebooks/test_output2/"

In [3]:
get_crops(df, base_path, output_path, bbox_col, chan1_col, chan2_col, chan3_col, 
          bbox_area=bbox_area, cluster_col=cluster_col)

0it [00:00, ?it/s]/hdd/tools/UMAP-Explorer/explorer/find_clusters.py:32: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  chan1 = io.imread(os.path.join(img_dir, row[chan1_col][0]))
/hdd/tools/UMAP-Explorer/explorer/find_clusters.py:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  chan2 = io.imread(os.path.join(img_dir, row[chan2_col][0]))
/hdd/tools/UMAP-Explorer/explorer/find_clusters.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos